In [5]:
# change working dir
import os
os.chdir('../')

import torch
from src.datasets.scannet import ScanNetDataset
from src.threedsam.backbone import build_backbone
from src.threedsam.threedsam_modules.feature_fusion import FeatureFusion

root_dir = '/media/morgen/T7/dataset/scannet_train'
npz_path = '/media/morgen/T7/dataset/scannet_indices/scene_data/train/scene0066_00.npz'
intrinsic_path = '/media/morgen/T7/dataset/scannet_indices/intrinsics.npz'
dpt_weight_path = '/home/morgen/projects/3DSAM/weights/dpt/dpt_hybrid_nyu-2ce69ec7.pt'
dataset = ScanNetDataset(root_dir, npz_path, intrinsic_path, dpt_weight_path)

/home/morgen/.conda/envs/3dsam/lib/python3.8/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name vit_base_resnet50_384 to current vit_base_r50_s16_384.orig_in21k_ft_in1k.
  model = create_fn(


In [6]:
# load a sample
sample = dataset[200]
sample.keys()

config = {
    'backbone_type': 'ResNetFPN',
    'resnetfpn': {
        'initial_dim': 64,
        'block_dims': [64, 128, 256, 512, 1024]
    }
}

backbone = build_backbone(config)
feat_fusion = FeatureFusion(config['resnetfpn'])

In [8]:
image0 = sample['image0'].unsqueeze(dim=0)
image1 = sample['image1'].unsqueeze(dim=0)

feat0_32, feat0_16, feat0_8, feat0_2 = backbone(image0)

feat_out8, feat_out16, feat_out8 = feat_fusion(feat0_8, feat0_16, feat0_32)